In [1]:
import torch
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Imagenet normalization
])

# Load the dataset
data_dir = "D:/SRIP/archive/animals/animals"
dataset = ImageFolder(root=data_dir, transform=transform)

# Split dataset into training and validation
train_size = int(0.1 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Define dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [3]:
print(dataset.classes)

['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra']


In [4]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
num_epochs = 10
num_classes = len(dataset.classes)

# Initialize a list to store trained models
trained_models = []

In [7]:
for i in range(num_classes):
    # Initialize the model
    model = CustomCNN(num_classes=2)  # Binary classification
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, (labels == i).long())  # Set target labels to 1 for the current class, 0 for others
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Class {i + 1}, Loss: {running_loss / len(train_loader)}")

    # Save trained model
    trained_models.append(model)

print("Training completed.")

Epoch [1/10], Class 1, Loss: 3.002804728115306
Epoch [2/10], Class 1, Loss: 0.21913853713639958
Epoch [1/10], Class 2, Loss: 0.5630387751431205
Epoch [2/10], Class 2, Loss: 0.12247658334672451
Training completed.


In [12]:
def predict_one_vs_rest(trained_models, image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Imagenet normalization
    ])
    
    image = image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension

    predictions = []
    for model in trained_models:
        model.eval()
        with torch.no_grad():
            output = model(image)
            predicted_class = torch.argmax(output).item()
            predictions.append(predicted_class)

    # Determine the most frequent predicted class
    predicted_class_index = max(set(predictions), key=predictions.count)

    return predicted_class_index

In [47]:
from PIL import Image
image_path = "D:/SRIP/archive/animals/animals/coyote/0a151f5c78.jpg"
predicted_class_index = predict_one_vs_rest(trained_models, image_path)
print("Predicted class index:", predicted_class_index)

Predicted class index: 0
